# Financial screener Data - ICA P2

It's a demo project to show possobilityes of [Plotly Dash](https://dash.plotly.com/)

## Introduction

For the ICA part 2, I decided to make application for screening and analyze financial data of currency exchange market. For my prospective one of the most demanded area of data science is Financial. The finance industry generates lots of data. Big data in finance refers to the petabytes of structured and unstructured data that can be used for creation strategies for business institutions.

In these parts I want to describe designe, cleaning data approach, and implement applicaion steps of screen market prices application.

Here some examples of the real screning services:
 - [finviz](https://finviz.com/)
 - [finance.yahoo](https://finance.yahoo.com/quote/%5EGSPC?p=%5EGSPC)
 - [investing](https://www.investing.com/equities/apple-computer-inc)

Also, in this project, I'll use some principles of technical analysis of the price charts and some special terminology. Anyway, I'll try to describe everything in simple language for a non-technical person. But, if you don't have any understanding of what price charts in the market are, you would find all the basic information in this article: [Understanding Basic Candlestick Charts](https://www.investopedia.com/trading/candlestick-charting-what-is-it/)

Since it's only a demo project, we will create something not complicated but with all main features that needed for real price screening application:

1. Charts with different display options
2. Slider with ability to change date range
3. Indicator - another chart with some displayed price analyse
4. Data frames with different currency pairs
5. Data frames with different timeframes
6. Possobility to retrieve data by year from data frame
7. \[Optional\] Dark and light theme for the chart.

Collecting all together, our application designe is:
![screener-designe](img/screener-designe.png)


## Data View

The most important thing to find a proper data. I desided to take a real financial data from [BITSTAMP EXCHANGE DATA](https://www.cryptodatadownload.com/data/bitstamp). For demo prospects I took only few currency pairs: <span style="color:orange">"BTCEUR"</span>, <span style="color:orange">"BTCUSD"</span>, <span style="color:orange">"ETHBTC"</span>, and <span style="color:orange">"ETHEUR"</span>. Each coin pair represended of two data frames: 1 day timeframe, and 1 hour timeframe. In total we have 8 data frames.

Let's import and take a look on one of the data frames:

In [82]:
import pandas as pd

df = pd.read_csv('data/BTCEUR_1h.csv')

df.head()

,,,,,,,,https://www.CryptoDataDownload.com
unix,date,symbol,open,high,low,close,Volume BTC,Volume EUR
1672790400,2023-01-04 00:00:00,BTC/EUR,15798,15802,15787,15788,1.65765029,26170.98277852
1672786800,2023-01-03 23:00:00,BTC/EUR,15783,15806,15783,15801,4.79777073,75809.57530473
1672783200,2023-01-03 22:00:00,BTC/EUR,15786,15789,15772,15783,3.24313136,51186.34225488
1672779600,2023-01-03 21:00:00,BTC/EUR,15775,15791,15770,15786,9.63655850,152122.712481


Looks like, we have a set of different prices and dates. We need to understand that every second and milisecond there are appeared a hunderd of the transactions in the real market, with every currency pair. It's impossible to show all of the price movement in the chart, but we don't need, it's quite enough to show the result of these transactions in the selected timeframe. For example let's get the firs row in our dataframe. We interested in the <span style="color:yellow">open</span> and <span style="color:yellow">close</span> values. Here we see that open is 15798 and close - 15788, the close is lower then open it mean's that we have down trend and the candle is red. The high of candle is difference beetween open and close. In other words we can say if difference si positive - it's a green candle, up trend, if negative - red candle, down trend.

Example of prices represended by candles:
![candle-example](img/candles-example.png)

The thin lines abowe and under candle body it's a range of price in the different timeframe, in another words lowest and highest price in the current timeframe. It's represented by  <span style="color:yellow">high</span> and  <span style="color:yellow">low</span> values in data frame.

Another important value it's date, we'll use it to manage order of candles in the chart

Another columns like <span style="color:yellow">unix</span>, <span style="color:yellow">Volume BTC</span>, <span style="color:yellow">Volume EUR</span> are not necassery for the chart view and we'll delete it during data cleaning

## Data cleaning

Before we jump into the cleaning process, let’s skip first row which is ad link and take a brief look at the data.

Before we jump into the cleaning process, let’s take one more look at the data set.

In [83]:
# Find all null values by columns
print(df.isnull().mean())

https://www.CryptoDataDownload.com    0.0
dtype: float64


In [84]:
# Total rows and columns
print(df.shape)

(40676, 1)


In [85]:
# Take a look on first rows
df.head()

,,,,,,,,https://www.CryptoDataDownload.com
unix,date,symbol,open,high,low,close,Volume BTC,Volume EUR
1672790400,2023-01-04 00:00:00,BTC/EUR,15798,15802,15787,15788,1.65765029,26170.98277852
1672786800,2023-01-03 23:00:00,BTC/EUR,15783,15806,15783,15801,4.79777073,75809.57530473
1672783200,2023-01-03 22:00:00,BTC/EUR,15786,15789,15772,15783,3.24313136,51186.34225488
1672779600,2023-01-03 21:00:00,BTC/EUR,15775,15791,15770,15786,9.63655850,152122.712481


In [86]:
# Take a look on last rows
df.tail()

,,,,,,,,https://www.CryptoDataDownload.com
1526378400,2018-05-15 10:00:00,BTC/EUR,7316.1,7427.72,7296.07,7381.92,707318.62,96.02
1526374800,2018-05-15 09:00:00,BTC/EUR,7313.2,7347.22,7300.0,7316.1,685687.14,93.52
1526371200,2018-05-15 08:00:00,BTC/EUR,7339.67,7339.67,7265.4,7313.2,343404.76,47.06
1526367600,2018-05-15 07:00:00,BTC/EUR,7335.0,7354.32,7306.34,7339.67,181611.86,24.79
1526364000,2018-05-15 06:00:00,BTC/EUR,7311.2,7390.7,7307.57,7335.0,480389.46,65.38


From these results, we learn that the dataset has 40675 rows and 9 columns. Of course it's only 1 hour timeframe dataset, for the day it would be less rows. We also identify that rows sorted by date in descent order. And we double checked that our data frame is already in consist state without null values. These are all useful information.

Now we can clean and transform data

#### Skip ad link rows

The first row in all dataframes is ad link. Let's drop it in the first place

In [92]:
df = pd.read_csv('data/BTCEUR_1h.csv', skiprows=1)

#### Cut unnecessary date

In total we have 40675 rows, and range of all date is from 2023 to 2018 year. I want to use a shorter range for the demo project. I want to take only 2019 - 2023 range

In [88]:
df = df[df['date'] > '2018-31-12']

#### Reverse data frame

Values of date column is ordered in the descent, which is not we need. We want to show price history in the chart chart from the oldest in the left to newest in the right conner of chart.

In [89]:
df = df.iloc[::-1]

#### Drop unnecessary columns

Also, as we mentioned before we don't need all columns in data frame to build chart. Let's left only necassery: date, open, close, high, low

In [90]:
df.drop(['unix', 'Volume {}'.format('BTCEUR'[:3]),
            'Volume {}'.format('BTCEUR'[3:6])], axis=1, inplace=True)